In [1]:
import tensorflow as tf
import keras

import os

from keras.utils import image_dataset_from_directory, load_img
from keras.optimizers import Adam

from keras import layers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#today
#DONE implement data augmentation
#DONE add class labels to image loading
#TODO streamline functions

#tmr
#TODO improve model overfitting (lower drop out)



In [42]:
img_shape = (48,48)
BS = 16
dir_train_images = 'data/Training'
dir_test_images = 'data/Test'
num_cat = 7

train_ds = image_dataset_from_directory(
    dir_train_images,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=img_shape,
    batch_size=BS,
    color_mode = 'grayscale',
    labels = 'inferred'
)

val_ds = image_dataset_from_directory(
    dir_train_images,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=img_shape,
    batch_size=BS,
    color_mode = 'grayscale',
    labels = 'inferred'

)

test_ds = image_dataset_from_directory(
    dir_test_images,
    seed=1337,
    image_size=img_shape,
    batch_size=BS,
    color_mode = 'grayscale',
    labels = 'inferred'
)

class_names = train_ds.class_names

Found 27463 files belonging to 7 classes.
Using 21971 files for training.
Found 27463 files belonging to 7 classes.
Using 5492 files for validation.
Found 6584 files belonging to 7 classes.


In [43]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=img_shape+(1,)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)


shallow_sequence = [
    #prepare data
    data_augmentation,
    layers.Rescaling(1./255),
    
    #layer 1
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"), #stride size = 1
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    #layer 2
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"), #stride size = 1
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    #FC layer
    layers.Flatten(),
    layers.Dense(512, activation='relu'),

    #logistic layer
    layers.Dense(num_cat, activation='softmax')

  ]

In [44]:
def facial_expr_model(sequence):
    
    model = keras.Sequential(sequence)
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [45]:
model = facial_expr_model(shallow_sequence)
model.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_27 (Sequential)  (None, 48, 48, 1)         0         
                                                                 
 rescaling_3 (Rescaling)     (None, 48, 48, 1)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 46, 46, 32)        320       
                                                                 
 batch_normalization_6 (Batc  (None, 46, 46, 32)       128       
 hNormalization)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 46, 46, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 44, 44, 64)        18496     
                                                     

In [46]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1
)

1374/1374 [==============================] - 546s 396ms/step - loss: 3.0255 - accuracy: 0.2714 - val_loss: 1.9417 - val_accuracy: 0.3003


In [38]:
def eval_model_stats(history):
    # Evaluate the model
    loss, acc = model.evaluate(test_ds, verbose=2)
    print("Accuracy: {:5.2f}%".format(100 * acc))

In [39]:
def plot_model_peformance(history, epochs):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

In [49]:
def save_model_weights(model, model_id):
    model.save(f'my_model_{model_id}', save_format='h5')

In [50]:
save_model_weights(model, 1)

In [37]:
def showcase_augmented_image(dataset, aug_sequence):
    plt.figure(figsize=(8, 8), facecolor='black')
    plt.suptitle("Example of an image augmented", fontsize=14, color = 'white')
    for image, _ in dataset.take(1):
        for i in range(9):
            augmented_images = aug_sequence(image)
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(augmented_images[0].numpy().astype("uint8"))
            plt.axis("off")


In [289]:
def predict_image(image_path, model):

    img = tf.keras.utils.load_img(
        image_path, target_size=img_shape, color_mode = 'grayscale'
        )

    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    return(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )